In [ ]:
# # Exemple d'avant :

# # Premier réseau de neuronnes : reconnaissances d'un chiffre écrit à la main :

# # Les deux jeux de données sont des jeu de données connues et utilisé pour apprendre les NN, on les téléchargent directement grâce au download=True, 
# #  On les téléchargent dans le répertoire du premier paramètre de MNIST ici : ''

# # Jeu de données pour l'apprentissage
# train = datasets.MNIST('', train=True, download=True,
#                        transform=transforms.Compose([
#                            transforms.ToTensor()
#                        ]))
# # Jeu de données pour les tests
# test = datasets.MNIST('', train=False, download=True,
#                        transform=transforms.Compose([
#                            transforms.ToTensor()
#                        ]))

# # On mélange les données et batch_size c'est combien de données à la fois on passe mais comme on va traiter des images de 28*28, nos CPU pouuraient tous traiter d'un coup
# trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
# testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)

# # La classe qui représente le réseau de neuronnes, elle hérites de nn.Module
# class Net(nn.Module):

# # On utilise init de nn.Module avec super()
#     def __init__(self):
#         super().__init__()
#         input = 28*28
#         output = output2 = output3 = 64
#         lastOutput = 10         # On veut reconnaître des chiffres donc la dernière sortie sera entre [0,9] donc de tailles 10

#         # On créer nos couches de neuronnes
#         self.fc1 = nn.Linear(input, output)         # Nb input = le nombre de valeur à prendre en compte pour la décision 
#                                                     # Nb output le nombre de choix possible pour une décision
#         self.fc2 = nn.Linear(output, output2)       # 1Deuxième couche de neuronnes
#         self.fc3 = nn.Linear(output2, output3)      # Troisième : Possibilité que output=output2=output3
#         self.fc4 = nn.Linear(output3, lastOutput)


# # Pour chaque données on l'a fait passer à travers notre réseau de neuronnes
#     def forward(self, x):
#         x = F.relu(self.fc1(x))         # F.relu transforme notre x (qui peut être = 15605 par ex) dans un intervalle [0,1]
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
#         x = self.fc4(x)                 # Pour notre dernière couche qui sera une matrice de taille [1, nb de coup jouable] 

#         return F.log_softmax(x, dim=1)  #On aura une matrice de taille 1,10 pour la reconnaissance des chiffres avec une probabilité que ce soit chacun des chiffres

# # Ici c'est mon main hihi
# net = Net()
# # print(net)

# # x = torch.rand((1,10))
# # print(x)
# # x = x.view(1,10)
# # output=net(x)
# # print(output)

# loss_function = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=0.001)      # lr = learning rate : permet de dire à l'optimizer de ne pas sur apprendre sur chaque données
#                                                         # On ne va pas que lorsqu'il se trompe dans sa prédiction, il se corrige pour avoir une probabilité de 0 partout et de 1 sur le bon chiffre
#                                                         # Sinon on va sur-apprendre et ce n'est pas bon
#                                                         # On va donc optimiser la perte "Loss" sans la rendre nulle pour éviter le sur-apprentissage
# EPOCHS = 3
# for epoch in range(EPOCHS): # 3 full passes over the data
#     for data in trainset:  # `data` is a batch of data
#          X, y = data  # X sont les données d'une image 28*28, y est le label càd le chiffre que représente l'image
#          net.zero_grad()    # On reset pour chaque data
#          output = net(X.view(-1,28*28))  # On fait passer notre data dans le NN (-1 pour faire ligne --> colonne)
#          loss = F.nll_loss(output, y)   # On calcul la perte avec l'output qu'on a eu et y le résultat voulu
#          loss.backward()                # Calcul tous seul le gradient (merci Pytorch)     
#          optimizer.step()               # Lance une étape d'optimisatin
#     print(loss)

# # On va voir à quel point on est correcte : 
# correct = 0
# total = 0

# with torch.no_grad():       # On ne veut pas des gradient ici on va juste regarder si on a bon ou pas sur la valeur avec le plus de proba
#     for data in testset:
#         X, y = data
#         output = net(X.view(-1,784))
#         #print(output)
#         for idx, i in enumerate(output):
#             #print(torch.argmax(i), y[idx])
#             if torch.argmax(i) == y[idx]:
#                 correct += 1
#             total += 1

# print("Accuracy: ", round(correct/total, 3))    # On arrondi la précision

# import matplotlib.pyplot as plt

# # Afficher la première image
# plt.imshow(X[0].view(28,28))
# plt.show()

# # Afficher notre prédiction (avec la proba la + élévé) de notre modèle
# print(torch.argmax(net(X[0].view(-1,784))[0]))

In [4]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image

print("Done")

Done


In [39]:
class DeepQNetwork(nn.Module):
    
    def __init__(self):
        super(DeepQNetwork, self).__init__()
        
        self.conv1 = nn.Sequential(nn.Linear(4, 64), nn.ReLU(inplace=True))     # Sequential : Linear and ReLu
        self.conv2 = nn.Sequential(nn.Linear(64, 64), nn.ReLU(inplace=True))    # Sequential : Linear and ReLu
        self.conv3 = nn.Sequential(nn.Linear(64, 1))
        self._create_weights()

    def _create_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)             # Loi uniforme
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x) # Au lieu de faire le RELU ici on le fait avant dans __init__
        x = self.conv2(x)
        x = self.conv3(x)

        return x
print("Done")

Done


In [20]:
# Fichier qui va entrainer notre réseau de neuronnes
import argparse
import os
import shutil
import tetris
import numpy as np


def get_args():
    # Paramètre de la grid Tetris
    parser = argparse.ArgumentParser(
        """Implementation of Deep Q Network to play Tetris""")
    # parser.add_argument("--width", type=int, default=10, help="The common width for all images")
    # parser.add_argument("--height", type=int, default=15, help="The common height for all images")
    # parser.add_argument("--block_size", type=int, default=30, help="Size of a block")
    # parser.add_argument("--batch_size", type=int, default=512, help="The number of images per batch")
    # parser.add_argument("--lr", type=float, default=1e-3)                   # Learning rate  : permet de dire à l'optimizer de ne pas sur apprendre sur chaque données
    #                                                                         # On ne va pas que lorsqu'il se trompe dans sa prédiction, il se corrige pour avoir une probabilité de 0 
    #                                                                         # partout et de 1 sur le bon chiffre
    #                                                                         # Sinon on va sur-apprendre et ce n'est pas bon.
    #                                                                         # On va donc optimiser la perte "Loss" sans la rendre nulle pour éviter le sur-apprentissage
    # parser.add_argument("--gamma", type=float, default=0.99)
    # parser.add_argument("--initial_epsilon", type=float, default=1)
    # parser.add_argument("--final_epsilon", type=float, default=0.001)
    # parser.add_argument("--num_decay_epochs", type=float, default=2000)
    # parser.add_argument("--num_epochs", type=int, default=3000)             # Nombre de génération
    # parser.add_argument("--save_interval", type=int, default=1000)          # Intervalle de sauvegarde du réseau
    # parser.add_argument("--replay_memory_size", type=int, default=30000,    
    #                     help="Number of epoches between testing phases")
    # parser.add_argument("--log_path", type=str, default="tensorboard")      # Chemin des logs
    # parser.add_argument("--saved_path", type=str, default="trained_models") # Chemin de sauvegarde
    parser.add_argument("--width", type=int, default=10, help="The common width for all images")
    parser.add_argument("--height", type=int, default=15, help="The common height for all images")
    parser.add_argument("--block_size", type=int, default=30, help="Size of a block")
    parser.add_argument("--batch_size", type=int, default=512, help="The number of images per batch")
    parser.add_argument("--lr", type=float, default=0.001)                   # Learning rate  : permet de dire à l'optimizer de ne pas sur apprendre sur chaque données
                                                                            # On ne va pas que lorsqu'il se trompe dans sa prédiction, il se corrige pour avoir une probabilité de 0 
                                                                            # partout et de 1 sur le bon chiffre
                                                                            # Sinon on va sur-apprendre et ce n'est pas bon.
                                                                            # On va donc optimiser la perte "Loss" sans la rendre nulle pour éviter le sur-apprentissage
    parser.add_argument("--gamma", type=float, default=0.99)
    parser.add_argument("--initial_epsilon", type=float, default=1)
    parser.add_argument("--final_epsilon", type=float, default=0.001)
    parser.add_argument("--num_decay_epochs", type=float, default=2000)
    parser.add_argument("--num_epochs", type=int, default=3000)             # Nombre de génération
    parser.add_argument("--save_interval", type=int, default=1000)          # Intervalle de sauvegarde du réseau
    parser.add_argument("--replay_memory_size", type=int, default=30000, help="Number of epoches between testing phases")
    parser.add_argument("--log_path", type=str, default="tensorboard")      # Chemin des logs
    parser.add_argument("--saved_path", type=str, default="trained_models") # Chemin de sauvegarde
    args = parser.parse_args()
    return args

# Fonction d'apprentissage
def train(opt):
    # Cuda permet d'éxécuter sur le GPU, si on peut le faire on l'utilise
    if torch.cuda.is_available():
        torch.cuda.manual_seed(123)
    # Sinon ça sera sur le CPU
    else:
        torch.manual_seed(123)
    # Récupération d'un log
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
    os.makedirs(opt.log_path)
    writer = SummaryWriter(opt.log_path)
    env = Tetris()          # Notre jeu tetris, ici c'est l'environnement de l'IA d'où "env"
    net = DeepQNetwork()    # Notre réseau de neuronnes
    optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr)     # Notre optimizer
    criterion = nn.MSELoss()        # Fonction de perte MeanSquar : va pénaliser le model pour de trop grande erreurs et encourage le model quand il en fait de petite

    state = env.reset()         # Remettre à zéro le tetris et retourner l'état 

    if torch.cuda.is_available():
        model.cuda()
        state = state.cuda()
epoch = 0
while epoch < opt.num_epochs :
    # next_steps = env.get_next_states()
    epsilon = opt.final_epsilon + (max(opt.num_decay_epochs - epoch, 0) * (
                opt.initial_epsilon - opt.final_epsilon) / opt.num_decay_epochs)
    u = random()
    random_action = u <= epsilon
print("Done")

NameError: name 'opt' is not defined

In [33]:
if __name__ == "__main__":
    print("OKKKOKOK")
    opt = get_args()
epsilon = opt.final_epsilon + (max(opt.num_decay_epochs - epoch, 0) * (
                opt.initial_epsilon - opt.final_epsilon) / opt.num_decay_epochs)
u = random()
random_action = u <= epsilon

usage: Implementation of Deep Q Network to play Tetris [-h] [--width WIDTH]
                                                       [--height HEIGHT]
                                                       [--block_size BLOCK_SIZE]
                                                       [--batch_size BATCH_SIZE]
                                                       [--lr LR]
                                                       [--gamma GAMMA]
                                                       [--initial_epsilon INITIAL_EPSILON]
                                                       [--final_epsilon FINAL_EPSILON]
                                                       [--num_decay_epochs NUM_DECAY_EPOCHS]
                                                       [--num_epochs NUM_EPOCHS]
                                                       [--save_interval SAVE_INTERVAL]
                                                       [--replay_memory_size REPLAY_MEMORY_SIZE]
                 

TypeError: object of type 'NoneType' has no len()